In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f'OpenAI API key is exist and begins {openai_api_key[:8]}')

else:
    print("OpenAI API key is not set.")

client = OpenAI()

OpenAI API key is exist and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI"
system_message += "Give short, courteous answers, no more than 1 sentence."
system_message += "Always be accurate. if you don't know the answer, say so."


In [4]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

#### Tools 
Tools are an incredibly powerful feature provided by the frontier LLMs.</br>
With tools, you cat write a function, and have the LLM call that function as part of its response.</br>
Sounds almost spooky.. we're giving it the power to run code on our machine?</br>
well, kinda

In [5]:
# Useful function

ticket_prices = {
    'london' : 100,
    'paris' : 150,
    'new York' : 500,
    'tokyo' : 800,
    'sydney' : 1000,
    'hong kong' : 600,
    'singapore' : 700,
    'dubai' : 400,
    'los angeles' : 550,
    'Toronto' : 450,
    'Berlin' : 200,
    'Madrid' : 180,
    'Rome' : 220,
    'Amsterdam' : 250,
    'Barcelona' : 230,
    'Moscow' : 300,
    'Istanbul' : 350,
    'Cairo' : 400,
    'Mumbai' : 600,
    'Bangkok' : 700,
    'Seoul' : 800
}


In [6]:
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    if city in ticket_prices:
        ticket = ticket_prices[city]
        ticket = f'$ {ticket}'
    else:
        ticket = "Sorry, I don't have the ticket price for that city."
    
    return ticket
     


In [7]:
get_ticket_price('London')

Tool get_ticket_price called for London


'$ 100'

In [8]:
price_function = {
    'name' : 'get_ticket_price',
    "description" : "Get the ticket price for a given destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is the ticket to this city'",
    "parameters" : {
        "type" : "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        'additionalProperties': False
    }
           
}

In [9]:
tools = [{"type": "function", "function": price_function}]

In [10]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [11]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/blocks.py", line 2146, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/blocks.py", line 1662, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/udemy_llm/l

In [64]:
import base64
from io import BytesIO
from PIL import Image

In [65]:
def artist(city):
    image_response = client.images.generate(
        model = 'dall-e-3',
        prompt = f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city} in a cubism art style",
        size = '1024x1024',
        n = 1,
        response_format='b64_json'
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    
    return Image.open(BytesIO(image_data))

In [66]:
image = artist('Seoul')
display(image)

KeyboardInterrupt: 

In [67]:
def chat(message, history):
    messages = [{'role': 'system', 'content' : system_message}]
    for human, assistant in history:
        messages.append({'role':'user', 'content': human})
        messages.append({'role':'assistant', 'content': assistant})
    
    messages.append({'role':'user', 'content': message})

    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        tools = tools # make use tools(price_functions)
    )

    if response.choices[0].finish_reason =="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = client.chat.completions.create(
            model = 'gpt-4o-mini',
            messages = messages
        )

    return response.choices[0].message.content

#### Audio

In [68]:
from pydub import AudioSegment
from pydub.playback import play


def talker(message):
    response = client.audio.speech.create(
        model = 'tts-1',
        voice = 'onyx',
        input = message
    )

    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format = 'mp3')
    play(audio)



    

In [69]:
talker("Hello I am a voice assistant. How can I help you?")

Input #0, wav, from '/var/folders/kh/6khx28zs2bq8wjptklpbkj440000gn/T/tmpxwtt6t4a.wav':
  Duration: 00:00:03.07, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s


### Agent Framework

1. Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
2. The ability for LLMs to use Tools to give them additional capabilities
3. The Agent Environment which allows Agents to collaborate
4. An LLM can act as the Planner dividing bigger tasks into smaller ones for the specialists
5. The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory.

We're showing 1 and 2 here, and to a lesser extent 3 and 5

In [70]:
def chat(message, history):
    image = None
    conversation = [{'role': 'system', 'content': system_message}]
    for human, assistant in history:
        conversation.append({'role': 'user', 'content': human})
        conversation.append({'role': 'assistant', 'content': assistant})

    conversation.append({'role': 'user', 'content': message})
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=conversation,
        tools=tools
    )

    if response.choices[0].finish_reason == "tool_calls":
        message = tool_call = response.choices[0].message
        response, city = handle_tool_call(message)

        conversation.append(message)
        conversation.append(response)

        image = artist(city)
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=conversation
        )

    reply = response.choices[0].message.content
    talker(reply)

    return reply, conversation, image



In [71]:
# More involved Gradio code as we're not using the preset Chat interface


with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height = 500)
        image_output = gr.Image(height = 500)

    with gr.Row():
        msg = gr.Textbox(label = 'Chat with our AI assistant :')
    
    with gr.Row():
        clear = gr.Button('Clear')

    def user(user_message, history):
        return "", history + [(user_message, None)]
    
    def bot(history):
        user_message = history[-1][0]
        bot_message, image = chat(user_message, history[:-1])
        history[-1][1] = bot_message

        return history, image
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue= False).then(
        bot, chatbot, [chatbot, image_output]
        )
    clear.click(lambda:None, None, chatbot, queue=False)

ui.launch()

/var/folders/kh/6khx28zs2bq8wjptklpbkj440000gn/T/ipykernel_88194/3308302509.py:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height = 500)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Input #0, wav, from '/var/folders/kh/6khx28zs2bq8wjptklpbkj440000gn/T/tmp7180w1dn.wav':
  Duration: 00:00:02.11, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
Traceback (most recent call last): 0KB vq=    0KB sq=    0B 
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/blocks.py", line 2146, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
   